<a href="https://colab.research.google.com/github/hiydavid/tfdev_learning/blob/main/ZTM/notebooks/tfdev_05_transfer_learning_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. Transfer Learning Fine Tuning

In this notebook we'll:

* Introducing transfer learning fine-tuning
* Introduce the Keras functional API
* Use a small dataset to experiment faster
* Data augmentation
* Running a series of experiments on our Food Vision data
* Introduce the ModelCheckpoint callback to save intermediate training results

# Import Helper Functions

In [2]:
# get prebuilt helper function 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-03-26 04:28:18--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-03-26 04:28:19 (45.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# import helper functions
from helper_functions import (
    create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir
)

# Get Data

In [7]:
# get data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2022-03-26 04:34:33--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 108.177.125.128, 142.250.157.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  49.1MB/s    in 3.3s    

2022-03-26 04:34:37 (49.1 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [8]:
# check out how many images and subdirectories
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ch

In [9]:
# create training / testing paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [12]:
# load data using image_dataset_from_directory
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE
)

test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE
)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [14]:
# check output, see that it is batches of data
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [15]:
# check class names
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [18]:
# check 1 sample from 1 batch
for images, labels in train_data_10_percent.take(1):
    print(images[0], labels[0])

tf.Tensor(
[[[167.71428  187.71428  186.71428 ]
  [168.9796   188.9796   187.9796  ]
  [166.9949   186.9949   185.9949  ]
  ...
  [173.29079  196.648    198.86226 ]
  [170.97449  194.97449  196.97449 ]
  [171.35718  195.35718  197.35718 ]]

 [[165.       185.       184.      ]
  [166.       186.       185.      ]
  [167.14285  187.14285  186.14285 ]
  ...
  [171.301    194.6582   196.87247 ]
  [170.07143  194.07143  196.07143 ]
  [171.35718  195.35718  197.35718 ]]

 [[166.63776  186.63776  185.63776 ]
  [163.12755  183.12755  182.12755 ]
  [163.30612  183.30612  182.30612 ]
  ...
  [171.2602   194.6174   196.83167 ]
  [171.42857  195.42857  197.42857 ]
  [173.78575  197.78575  199.78575 ]]

 ...

 [[157.45413   29.224169  49.162952]
  [176.90744   85.54449  104.23338 ]
  [212.15355  175.16818  191.73468 ]
  ...
  [ 99.35199   15.367578  22.683788]
  [113.92923   66.89916   61.64383 ]
  [128.86209  117.20462   97.92373 ]]

 [[177.81686   85.9449    97.25096 ]
  [144.2704    36.48995   

# Build Model with `tf.keras.applications`